In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sentiment Analysis Model on Vertex AI with Hugging Face 🤗

## Overview

This project showcases building and deploying a text sentiment classification model. The model is developed by finetuning [BERT][bert], a popular NLP foundational model from [Hugging Face][hf]. To facilitate model development, deployment, and management, the project leverages various services including [Vertex AI][vertex], [Pytorch SDK][pytorch], [Artifact Registry][ar], [Cloud Build][cb], [Cloud Deploy][cd] and [Cloud Storage][cs]. 

[bert]: https://huggingface.co/bert-base-cased
[hf]: https://huggingface.co/
[vertex]: https://cloud.google.com/vertex-ai
[ar]: https://cloud.google.com/artifact-registry
[cb]: https://cloud.google.com/build
[cd]: https://cloud.google.com/deploy
[cs]: https://cloud.google.com/storage
[pytorch]: https://pytorch.org/

### Objective

In this tutorial, you learn to build, train, tune and deploy a PyTorch model on [Vertex AI](https://cloud.google.com/vertex-ai). You mainly focus on support for custom model training and deployment on Vertex AI. 


This tutorial uses the following Google Cloud services:

- MLOps
  - Vertex AI Workbench
  - Vertex AI Training
  - Vertex AI Model Registry
  - Vertex AI Predictions
- DevOps
  - Cloud Build
  - Cloud Deploy
  - Artifact Registry
  - Cloud Storage

The steps performed include:

- Create training package for the text classification model.
- Train the model with custom training on Vertex AI.
- Check the created model artifacts.
- Create a custom container for predictions.
- Deploy the trained model to a Vertex AI Endpoint using the custom container for predictions.
- Send online prediction requests to the deployed model and validate.
- Clean up the resources created in this notebook.

### Dataset

The dataset used for this tutorial is the [Happy Moments](https://www.kaggle.com/ritresearch/happydb) dataset from Kaggle. Learn more about the dataset in [HappyDB](https://rit-public.github.io/HappyDB/).

### Costs 

Learn about pricing for [Vertex AI](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage](https://cloud.google.com/storage/pricing), [Cloud Build](https://cloud.google.com/build/pricing), [Cloud Deploy](https://cloud.google.com/deploy/pricing) and [Artifact Registry](https://cloud.google.com/artifact-registry/pricing). Use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the packages required for executing this notebook.

In [ ]:
! pip3 install google-cloud-aiplatform accelerate transformers --upgrade

#### Set your project ID

If you know your project ID, update the `PROJECT_ID` variable.

In [ ]:
PROJECT_ID = "<your_project_id>"  # @param {type:"string"}

Otherwise, get the project ID using gcloud.

In [ ]:
if (
    PROJECT_ID == ""
    or PROJECT_ID is None 
    or PROJECT_ID == "<your_project_id>"
):
    import google.auth

    _, PROJECT_ID = google.auth.default()
    print("Project ID: ", PROJECT_ID)

Set the project ID in your active configuration.

In [ ]:
! gcloud config set project {PROJECT_ID}

#### Set the region

Set the `REGION` variable; it defaults to `"us-central1"`. Learn more about Vertex AI [regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [2]:
REGION = "us-central1"  # @param {type: "string"}

#### UUID

To avoid name collisions, generate a UUID and append it to resources created in this notebook.

In [3]:
import random
import string

def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))

UUID = generate_uuid()
print("UUID: ", UUID)

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [4]:
BUCKET_URI = f"gs://sentimeter-ml-artifacts-{UUID}"  # @param {type:"string"}

Create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### Import libraries and define constants

Import the required libraries for this notebook.

In [6]:
import base64
import json
import os
import sys
import accelerate
import transformers

from google.cloud import aiplatform
from google.protobuf.json_format import MessageToDict

Define the constants needed for this project.

In [7]:
# Name for the package application / model / repository
APP_NAME = "finetuned-bert-classifier"

# URI for the pre-built container for custom training
PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-11:latest"
)

# Name of the folder where the python package needs to be stored
PYTHON_PACKAGE_APPLICATION_DIR = "pkg"

# Path to the source distribution tar of the python package
SOURCE_PACKAGE_FILE_NAME = f"{PYTHON_PACKAGE_APPLICATION_DIR}/dist/trainer-0.1.tar.gz"

# GCS path where the python package is stored
PYTHON_PACKAGE_GCS_URI = (
    f"{BUCKET_URI}/pytorch-on-gcp/{APP_NAME}/train/python_package/trainer-0.1.tar.gz"
)

# Module name for training application
PYTHON_MODULE_NAME = "trainer.task"

# Training job's display name
JOB_NAME = f"{APP_NAME}-pytorch-pkg-train-{UUID}"

# Set training job's machine-type
TRAIN_MACHINE_TYPE = "n1-standard-8"

# Training job's accelerator type
# One of _ACCELERATOR_TYPE_UNSPECIFIED_, _NVIDIA_TESLA_K80_, _NVIDIA_TESLA_P100_, 
# _NVIDIA_TESLA_V100_, _NVIDIA_TESLA_P4_, _NVIDIA_TESLA_T4_, _NVIDIA_TELSA_A100_
TRAIN_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"

# The number of accelerators to attach to a worker replica for the training job
TRAIN_ACCELERATOR_COUNT = 1

# The number of worker replicas
REPLICA_COUNT = 1

TRAINING_ARGS = ["--num-epochs", "2", "--model-name", APP_NAME]

# The name of the container image for prediction
CUSTOM_PREDICTOR_IMAGE_URI = (
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{APP_NAME}/pytorch_predict_{APP_NAME}:latest"
)

# The version for model-deployment
VERSION = 1

# The model display name and id
MODEL_DISPLAY_NAME = f"{APP_NAME}-v{VERSION}"
MODEL_ID = f"{APP_NAME}-v{VERSION}"

# The model description
MODEL_DESCRIPTION = "PyTorch based text classifier with custom container"

# The health route for prediction container
HEALTH_ROUTE = "/ping"

# The predict route for prediction container
PREDICT_ROUTE = f"/predictions/{APP_NAME}"

# The serving container ports for prediction
SERVING_CONTAINER_PORTS = [7080]

# The display name for endpoint
ENDPOINT_DISPLAY_NAME = f"{APP_NAME}-endpoint"

# The machine-type for deployment
DEPLOY_MACHINE_TYPE = "n1-standard-4"

### Initialize the Vertex AI SDK for Python

In [8]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## Custom Training on Vertex AI

The `pkg` directory structure uses the packaging approach [recommended](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container#structure) by Vertex AI.

* The main directory contains a `setup.py` file with the required dependencies.
* The `trainer` directory contains:
  * `experiment.py` - Runs the model training and evaluation experiment, and exports the final model.
  * `metadata.py` - Defines the metadata for classification tasks such as predefined model, dataset name and target labels.
  * `model.py` -  Includes a function to create a model with a sequence classification head from a pre-trained model.
  * `task.py` - Main application module initializes and parse task arguments and hyperparameters. It also serves as an entry point to the trainer.
  * `utils.py` - Includes utility functions such as those used for reading data, saving models to Cloud Storage buckets.
  * `__init__.py` - Indicates to [Python Setuptools](https://setuptools.readthedocs.io/en/latest/) to include all subdirectories of the parent directory as dependencies. 

Create a source distribution.

In [ ]:
! cd {PYTHON_PACKAGE_APPLICATION_DIR} && python3 setup.py sdist --formats=gztar

Upload the source distribution with the training application to Cloud Storage bucket.

In [ ]:
! gsutil cp {SOURCE_PACKAGE_FILE_NAME} {PYTHON_PACKAGE_GCS_URI}

Validate that the source distribution exists in the Cloud Storage bucket.

In [ ]:
! gsutil ls -l {PYTHON_PACKAGE_GCS_URI}

### Run a custom job in Vertex AI using a pre-built container 

You don't need to build a PyTorch environment from scratch for running the training application because Vertex AI provides [pre-built containers](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#available_container_images) which are Docker container images that you can use for custom training. They include some common dependencies used in training code based on the machine learning framework and framework version. Learn more about [CustomPythonPackageTrainingJob](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomPythonPackageTrainingJob) which is used to launch a Custom Training Job in Vertex AI using a Python package.

Configure a [Custom Job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job) with the [pre-built container for PyTorch](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#pytorch) and training code packaged as Python source distribution. 

In [20]:
job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=JOB_NAME,
    python_package_gcs_uri=PYTHON_PACKAGE_GCS_URI,
    python_module_name=PYTHON_MODULE_NAME,
    container_uri=PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI,
)

Run the Custom training job. Note that the training may take over 24 hours.

In [ ]:
model = job.run(
    replica_count=REPLICA_COUNT,
    machine_type=TRAIN_MACHINE_TYPE,
    accelerator_type=TRAIN_ACCELERATOR_TYPE,
    accelerator_count=TRAIN_ACCELERATOR_COUNT,
    args=TRAINING_ARGS,
)

Validate that the model artifacts are written to Cloud Storage by the training code after the job completes successfully.

In [ ]:
job_response = MessageToDict(job._gca_resource._pb)
GCS_MODEL_ARTIFACTS_URI = job_response["trainingTaskInputs"]["baseOutputDirectory"]["outputUriPrefix"]
print(f"Model artifacts are available at {GCS_MODEL_ARTIFACTS_URI}")

! gsutil ls -lr $GCS_MODEL_ARTIFACTS_URI/

## Deployment

Deploying a PyTorch [model on Vertex AI](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions) requires a custom container that serves online predictions on a Vertex AI Endpoint. Deploy a container running [PyTorch's TorchServe](https://pytorch.org/serve/) tool in order to serve predictions from the fine-tuned transformer model for a sentiment analysis task. Then, use Vertex AI's online prediction service to classify the sentiment of input texts. Learn more in the deployment [docs](docs/deployment.md).

### Create a custom container image to serve predictions

Use [Cloud Build](https://cloud.google.com/build) to create the custom container image and store it in [Artifact Registry](https://cloud.google.com/artifact-registry).

#### Download the model artifacts from GCS to local directory

First, validate that model artifact files exist in the Cloud Storage bucket.

In [ ]:
! gsutil ls -r $GCS_MODEL_ARTIFACTS_URI/model/

Then, copy the files from Cloud Storage to a local directory.

In [ ]:
! gsutil -m cp -r $GCS_MODEL_ARTIFACTS_URI/model/ ./predictor/

Finally, validate that the model artifacts were copied to the local directory.

In [ ]:
! ls -ltrR ./predictor/model

#### Dockerfile for the image

The [Dockerfile](predictor/Dockerfile) is based on the TorchServe and performs several steps:

 - Install dependencies.
 - Add model artifacts.
 - Add the custom handler script.
 - Define the serving configuration e.g. health and prediction listener ports.
 - Run the Torch model archiver to create a model archive file from the files copied into in the container image.
 - Launch TorchServe HTTP server, which references the configuration properties and allows serving for the model.

Replace the `APP_NAME` variable in the `Dockerfile` with the value specified above.

In [ ]:
! sed -i- "s/\$APP_NAME/${APP_NAME}/g" "Dockerfile" && rm Dockerfile-

#### Create a docker repository in Artifact Registry


Create a new Docker repository in Artifact Registry with your specified region and description. Set `APP_NAME` as the name of the repository.

In [ ]:
! gcloud artifacts repositories create {APP_NAME} \
    --repository-format=docker \
    --location={REGION} \
    --description="Docker repo for ML"

List all Artifact Registry repositories and check for the new repository.

In [ ]:
! gcloud artifacts repositories list

#### Build the docker image using Cloud Build

Build a docker image inside the created repository using Cloud Build. Cloud Build tries to locate the repository path provided in the tag.

Learn more about building and pushing a docker image with [Cloud Build](https://cloud.google.com/build/docs/build-push-docker-image).

In [ ]:
! gcloud builds submit ./predictor \
--region={REGION} \
--tag=$CUSTOM_PREDICTOR_IMAGE_URI

### Deploy the serving container to Vertex AI

Deploying a model to a Vertex AI Endpoint is necessary for making online predictions. This process involves creating a model resource in the Vertex AI Model Registry and deploying the model to the endpoint. The deployed model then runs a custom container image to serve predictions.

#### Upload the model and image to Vertex AI Model Registry

Create a Vertex AI model resource with the created model artifacts and the container image in Vertex AI Model Registry.

In [ ]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    model_id=MODEL_ID,
    description=MODEL_DESCRIPTION,
    serving_container_image_uri=CUSTOM_PREDICTOR_IMAGE_URI,
    serving_container_predict_route=PREDICT_ROUTE,
    serving_container_health_route=HEALTH_ROUTE,
    serving_container_ports=SERVING_CONTAINER_PORTS,
)

model.wait()

print("Model display name: ", model.display_name)
print("Model resource name: ",model.resource_name)

#### Create a Vertex AI Endpoint

Create a Vertex AI Endpoint to deploy the registered Vertex AI model.

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name=ENDPOINT_DISPLAY_NAME)
ENDPOINT_ID = endpoint.name

print("Endpoint: ", endpoint)
print("Endpoint ID: ", ENDPOINT_ID)

#### Deploy the Model to Endpoint using Cloud Deploy

[Cloud Deploy](https://cloud.google.com/deploy) makes continuous delivery of models easy by allowing users to define releases and progress them through environments such as test, stage, and production. It provides easy promotion, approval and rollback of releases. 

We can use Cloud Deploy to deploy the model to a target endpoint. The `build_and_register.sh` builds the Vertex AI model deployer image and registers a Cloud Deploy custom target type that references the image. The script executes these steps:
* Define a custom action in the  file, which is similar to deploy hooks.
* Define a custom target type, which is a Cloud Deploy resource identifying the custom action used by targets of this type.
* Set up a target definition for a custom target, which is similar to any target type but includes additional properties.
* Set up a Cloud Deploy delivery pipeline that references the configured target.

Clone the Cloud Deploy samples repository, set the current directory to the vertex AI sample quickstart folder, then run the script.

In [ ]:
! git clone https://github.com/googlecloudplatform/cloud-deploy-samples.git

! cd cloud-deploy-samples/custom-targets/vertex-ai/quickstart

! ../build_and_register.sh -p $PROJECT_ID -r $REGION

Fill in the placeholders in the Cloud Deploy and Skaffold configuration values with the actual images.

In [ ]:
TMPDIR="tmp"

! mkdir -p $TMPDIR

! ./replace_variables.sh -p $PROJECT_ID -r $REGION -e $ENDPOINT_ID -t $TMPDIR

Apply the Cloud Deploy configuration defined in `clouddeploy.yaml`.

In [ ]:
! gcloud deploy apply --file=$TMPDIR/clouddeploy.yaml --project=$PROJECT_ID --region=$REGION

Create a release and rollout.

In [ ]:
RELEASE_NAME = "release-0001"
DELIVERY_PIPELINE_NAME = "vertex-ai-cloud-deploy-pipeline"
DEPLOY_PARAMS = f'customTarget/vertexAIModel=projects/{PROJECT_ID}/locations/{REGION}/models/{MODEL_ID}'

! gcloud deploy releases create $RELEASE_NAME \
    --delivery-pipeline=$DELIVERY_PIPELINE_NAME \
    --project=$PROJECT_ID \
    --region=$REGION \
    --source=$TMPDIR/configuration \
    --deploy-parameters=$DEPLOY_PARAMS


Monitor the release's progress.

In [ ]:

! gcloud deploy releases describe $RELEASE_NAME \
    --delivery-pipeline=$DELIVERY_PIPELINE_NAME \
    --project=$PROJECT_ID \
    --region=$REGION

Monitor the rollout status.

In [ ]:
TARGET_NAME = "prod-endpoint"

! gcloud deploy rollouts describe $(gcloud deploy targets describe $TARGET_NAME --delivery-pipeline=$DELIVERY_PIPELINE_NAME --region=$REGION --format="value('Latest rollout')") \
    --release=$RELEASE_NAME \
    --delivery-pipeline=$DELIVERY_PIPELINE_NAME \
    --project=$PROJECT_ID \
    --region=$REGION

## Send online prediction requests

Now, invoke the endpoint where the model is deployed using the Vertex AI SDK to make predictions for some test instances.

### Define and format input for online prediction

This notebook uses [TorchServe KServe](https://pytorch.org/serve/inference_api.html#kserve-inference-api) based inference API, which is also a Vertex AI predictions [compatible format](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements#prediction). For online prediction requests, format the prediction input instances as JSON with base64 encoding as follows:

```
[
    {
        "data": {
            "b64": "<base64 encoded string>"
        }
    }
]
```

In [ ]:
test_instances = [
    b"I went to a meeting that went really well.",
    b"I ran four miles this morning with a good time.",
    b"Watching the storms we had yesterday.  The lightning was incredible!",
    b"The last night I said with her 'I love you '. And she said ' Yes'.",
    b"I had followed a complex recipe making roasted duck, which took me hours and I had successfully made it.",
    b"I woke up this morning to birds chirping.",
]

formatted_test_instances = []
for test_instance in test_instances:
    b64_encoded = base64.b64encode(test_instance)
    formated_test_instance = [{"data": {"b64": f"{str(b64_encoded.decode('utf-8'))}"}}]
    formatted_test_instances.append(formated_test_instance)

print(formatted_test_instances)

### Send online prediction requests

Call prediction endpoint with formatted input requests and get the responses.

In [ ]:
for i in range(len(test_instances)):
    test_instance = test_instances[i]
    formatted_test_instance = formatted_test_instances[i]
    prediction = endpoint.predict(instances=formatted_test_instance)
    
    print(f"Input: \n\t{test_instance.decode('utf-8')}\n")
    print(f"Formatted Input: \n{json.dumps(formated_test_instance, indent=4)}\n")
    print(f"Prediction Response: \n\t{prediction}")
    print("=" * 100)

## Cleaning up 

To clean up all Google Cloud resources used in this notebook, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Vertex AI Custom Training Job
- Vertex AI Model
- Vertex AI Endpoint
- Cloud Storage Bucket
- Artifact Registry Repository

In [ ]:
# Set to true if you want to delete the bucket
delete_bucket = False

# Delete the Custom training job
job.delete()

# Undeploy the model from the endpoint
endpoint.undeploy_all()
# Delete the endpoint
endpoint.delete()

# Delete the Vertex AI Model resource
model.delete()

# Delete the Cloud Storage bucket
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI

# Delete artifact repository
! gcloud artifacts repositories delete $APP_NAME --location=$REGION --quiet

## Acknowledgements

This notebook is inspired by the Hugging Face [Token Classification](https://github.com/huggingface/notebooks/blob/9d8acb94105649c03ad6ce1c7f702520100fc41b/examples/token_classification.ipynb), [Run Glue](https://github.com/huggingface/transformers/blob/fb560dcb075497f61880010245192e7e1fdbeca4/examples/run_glue.py), [Text Classification](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/39da99a8bbe57184199e79dcb58ef8dc23cf54ff/notebooks/official/training/pytorch-text-sentiment-classification-custom-train-deploy.ipynb), and [Vertex AI Model Deployer](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/39da99a8bbe57184199e79dcb58ef8dc23cf54ff/notebooks/community/model_registry/get_started_with_vertex_ai_deployer.ipynb) notebooks.